In [1]:
import epidag as dag
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
scr = '''
PCore test {
    prob ~ beta(1, 1)
    x ~ binom(n, prob)    
}
'''

bn = dag.bayes_net_from_script(scr)

In [3]:
from epidag.fitting import Fitter, BayesianModel


In [33]:
class BinBeta(BayesianModel):
    def __init__(self, bn, data):
        BayesianModel.__init__(self, bn, pars=['prob'])
        self.Data = data
        
    @property
    def has_exact_likelihood(self):
        return True

    def evaluate_distance(self, pars):
        return - self.evaluate_likelihood(pars)

    def evaluate_likelihood(self, pars):
        li = 0
        pars = dict(pars)
        for d in self.Data:
            pars.update(d)
            li += self.BN['x'].evaluate(pars)
        return li



In [34]:
data = [
    {'id': 1, 'n': 10, 'x': 4},
    {'id': 2, 'n': 20, 'x': 7}
]

model = BinBeta(bn, data)


In [35]:
p = model.sample_prior()
model.evaluate_distance(p), model.evaluate_likelihood(p)

(22.433845064673246, -22.433845064673246)

In [36]:
p = {'prob': 11/30}
model.evaluate_distance(p), model.evaluate_likelihood(p)

(3.1093340676380965, -3.1093340676380965)

In [37]:
alg = dag.fitting.ABC(p_test = 0.05)

res = alg.fit(model, n_post = 200)

INFO:ABC:Testing threshold
INFO:ABC:Detected epsilon = -3.144
INFO:ABC:Collecting posterior parameters
INFO:ABC:Completed


In [31]:
import numpy as np

n_test = 100
p_test = 0.1

tests = list()

while len(tests) < n_test:
    p = model.sample_prior()
    li = model.evaluate_likelihood(p)
    if np.isfinite(li):
        tests.append(li)

eps = np.percentile(tests, (1 - p_test) * 100)
eps

36.39465378557255

In [45]:
res.to_df().describe()

,LogLikelihood,prob
count,200.000000,200.000000
mean,-3.120921,0.365412
std,0.009800,0.013349
min,-3.143159,0.343996
25%,-3.127818,0.352795
50%,-3.119437,0.364962
75%,-3.111715,0.378026
max,-3.109337,0.389432


In [38]:
fit = dag.fitting.MCMC(dm)
fit.fit(1000)
fit.summarise_posterior()

AttributeError: module 'epidag.fitting' has no attribute 'MCMC'

In [15]:
fit = dag.fitting.ABC(dm)
fit.fit(100, test_p=0.1)
fit.summarise_posterior()

INFO:root:Initialising
INFO:root:Testing threshold
INFO:root:Fitting
INFO:root:Gathering posteriori
INFO:root:Finished
          al     be        prob
count  100.0  100.0  100.000000
mean     1.0    1.0    0.366832
std      0.0    0.0    0.034689
min      1.0    1.0    0.311369
25%      1.0    1.0    0.334778
50%      1.0    1.0    0.367846
75%      1.0    1.0    0.397980
max      1.0    1.0    0.424331
N       100.000000
ESS      99.575397
Dbar      6.373474
pD        0.004413
DIC       6.377887
dtype: float64


In [9]:
scr = '''
PCore ex {
    sun ~ unif(0, 1)
    b0 = 1
    heat ~ exp(sun)
    x1 ~ norm(0, 1)
    x2 ~ norm(0, 1)
    b1 ~ unif(10, 20)
    mu1 = b0 + b1*x1 + b2*x2
    mu2 = b0 + b1*x1
    y1 ~ norm(mu1, sd)
    y2 ~ norm(mu2, 1)
}

'''

bn = dag.bayes_net_from_script(scr)
hie = {
    'g': ['sun', 'agent'],
    'agent': ['x1', 'x2','mu1', 'mu2']
}
sm = dag.as_simulation_core(bn, hie=hie, random=['x2'])


h = dag.form_hierarchy(bn, hie)

h.print()

g(heat, b2, sd, b1, b0, sun)
-- agent(mu1, y2, x1, mu2, y1, x2)


In [10]:
pc = sm.generate(exo={'sd'})
ag1 = pc.breed('a1', 'agent')

In [19]:
ag1.get_sampler('y1')(b2= 4, sd =3)

-1.5826879957652007

In [13]:
pc.get_sampler('heat')()

0.7308904995335411

In [12]:
ng = dag.form_hierarchy(bn, hie)
dag.analyse_node_type(bn, ng, True)


Group g
Exogenous    : ['b2', 'sd']
Can be fixed: ['b1', 'b0', 'sun', 'heat']
Can be random: ['sun', 'heat']
Can be actors: ['heat']
--Group agent
  Exogenous    : []
  Can be fixed: ['x1', 'mu2', 'x2', 'y2']
  Can be random: ['x1', 'mu2', 'x2', 'y2', 'y1', 'mu1']
  Can be actors: ['y2', 'y1']


{'g': (['b2', 'sd'], ['b1', 'b0'], ['sun'], ['heat'], [], []),
 'agent': ([], [], ['x1', 'mu2', 'x2'], ['y2'], ['y1'], ['mu1'])}